In [1]:
!pip install datamol
!pip install rdkit-pypi
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 52.3 MB/s eta 0:00:00


In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Linear
from torch.utils.data import DataLoader
import tqdm
from scipy.stats import spearmanr

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Project/ALVS")

Mounted at /content/drive


In [4]:
def ecfp_from_smiles(smiles,
                     R = 2,
                     L = 2**11,
                     use_features = True,
                     use_chirality = False):
    """
    Inputs:

    - smiles ... SMILES string of input compound
    - R ... maximum radius of circular substructures
    - L ... fingerprint-length
    - use_features ... if false then use standard DAYLIGHT atom features, if true then use pharmacophoric atom features
    - use_chirality ... if true then append tetrahedral chirality flags to atom features

    Outputs:
    - np.array(feature_list) ... ECFP with length L and maximum radius R
    """

    molecule = Chem.MolFromSmiles(smiles)
    if not molecule:
        return np.nan
    feature_list = AllChem.GetMorganFingerprintAsBitVect(molecule,
                                                         radius = R,
                                                         nBits = L,
                                                         useFeatures = use_features,
                                                         useChirality = use_chirality)
    return np.array(feature_list)

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, fps, scores):
        self.fps = fps
        self.scores = scores
        self.dataLen = len(scores)

    def __getitem__(self, index):
        fp = np.array(self.fps[index])
        score = np.array(self.scores[index]).reshape(1,)
        return fp, score

    def __len__(self):
        return self.dataLen

In [6]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = Linear(2048, 1024)
        self.dropout1 = nn.Dropout(p=0.2)
        self.fc2 = Linear(1024, 516)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = Linear(516, 256)
        self.dropout3 = nn.Dropout(p=0.2)
        self.fc4 = Linear(256, 128)
        self.dropout4 = nn.Dropout(p=0.2)
        self.fc5 = Linear(128, 1)


    def forward(self, data):
        x = F.relu(self.fc1(data))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = F.relu(self.fc4(x))
        x = self.dropout4(x)
        x = self.fc5(x)
        return x

In [7]:
def train_epoch(loader, model, optimizer, device):
    model.train()

    loss_all = 0
    i = 0
    for fps, scores in loader:
        fps = fps.float().to(device)
        scores = scores.float().to(device)

        optimizer.zero_grad()
        output = model(fps)
        loss = F.mse_loss(output, scores)
        loss.backward()

        loss_all += loss.item()
        optimizer.step()
        i += 1
    return loss_all / i

In [8]:
def test_epoch(loader, model,  device):
    model.eval()

    MSE, MAE = 0, 0
    trues, preds = [], []
    with torch.no_grad():
        for fps, scores in loader:
            fps = fps.float().to(device)
            scores = scores.float().to(device)

            output = model(fps)
            pred = output.cpu().squeeze().numpy().tolist()
            true = scores.cpu().squeeze().numpy().tolist()

            trues.extend(true)
            preds.extend(pred)
    MAE = mean_absolute_error(trues, preds)
    RMSE = np.sqrt(mean_squared_error(trues, preds))
    R2 = r2_score(trues, preds)
    Sp = spearmanr(trues, preds)[0]
    return MAE, RMSE, R2, Sp

In [9]:
def init_model(device, lr=0.0001):
    model = DNN()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    return model, optimizer

In [10]:
def prepare_dataloader(dff, batch_size, test_size=0.8):
    train_fps, valid_fps, train_scores, valid_scores = train_test_split(
                                                                        dff["ecfp"],
                                                                        dff["score"],
                                                                        test_size=test_size)
    train_dataset = Dataset(fps=list(train_fps),
                            scores = list(train_scores))
    valid_dataset = Dataset(fps=list(valid_fps),
                            scores = list(valid_scores))
    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=batch_size,
                              shuffle=False)
    return train_loader, valid_loader

In [11]:
def train_step(dff, epochs, batch_size, step_time, device="cuda"):
    model, optimizer = init_model(device=device)
    train_loader, valid_loader = prepare_dataloader(dff,  batch_size=batch_size)

    model_folder = "models/dpp4/ecfp/models_random/{}".format(step_time)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)

    hist = {"train-loss":[], "test-mae":[], "test-rmse":[], "test-r2":[],"test-sp":[]}
    for epoch in range(epochs):
        train_loss = train_epoch(train_loader, model, optimizer, device)
        test_mae, test_rmse, test_r2, test_sp = test_epoch(valid_loader, model, device)
        hist["train-loss"].append(train_loss)
        hist["test-mae"].append(test_mae)
        hist["test-rmse"].append(test_rmse)
        hist["test-r2"].append(test_r2)
        hist["test-sp"].append(test_sp)

        if test_rmse <= min(hist["test-rmse"]):
            weight_path = os.path.join(model_folder, "weight_{}.pth".format(epoch))
            torch.save(model.state_dict(), weight_path)

        # print(f'Epoch: {epoch}, Train loss: {train_loss:.3}, Test mae: {test_mae:.3}, Test rmse: {test_rmse:.3}, Test r2: {test_r2:.3}')
    print("---------------------------------\nvalidation min rmse: {}\n---------------------------------\n".format(min(hist["test-rmse"])))
    return weight_path

In [12]:
def random_select(df, n_samples):
    df_select = df.sample(n=n_samples).copy()
    sample_indexs = df_select["name"].tolist()
    df_remaining = df[~df["name"].isin(sample_indexs)].copy()
    return df_select, df_remaining

In [13]:
def predict(fpnp, model, device):
    fpnp = fpnp.reshape(1, 2048)
    fp = torch.from_numpy(fpnp).float().to(device)
    with torch.no_grad():
        output = model(fp)
        pred = output.cpu().numpy()[0][0]
    return pred

In [14]:
def load_model(best_model_path, device="cuda", enable_dp=True):
    model= DNN().to(device)
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()
    if enable_dp:
        enable_dropout(model)
    return model

In [15]:
def eval_model(dff, best_model_path):
    model = load_model(best_model_path, enable_dp=False)
    dff["pred"] = dff["ecfp"].map(lambda x: predict(x,  model, device="cuda"))
    MAE = mean_absolute_error(dff["score"], dff["pred"])
    RMSE = np.sqrt(mean_squared_error(dff["score"], dff["pred"]))
    R2 = r2_score(dff["score"], dff["pred"])
    Sp = spearmanr(dff["score"], dff["pred"])[0]
    return MAE, RMSE, R2, Sp

In [16]:
def run_step(dfs, epochs, stime):
    dfs_new = dfs.copy()

    best_model_path = train_step(dfs_new, epochs, batch_size=32, step_time=stime)
    return best_model_path

In [17]:
def run_iterations(df, df_test, epochs=200, iterations=100, n_samples=100):
    results = {"RMSE": [], "R2": [], "Sp": []}

    print("len(df):", len(df))

    df["ecfp"] = df["smiles"].map(lambda x: ecfp_from_smiles(x))
    df = df[~df["ecfp"].isna()]

    for stime in range(iterations):
        if stime < 1:
            dfs, dfr = random_select(df, n_samples)
        else:
            df_temp = dfs.copy()
            dfs, dfr = random_select(dfr, n_samples)
            dfs = pd.concat([dfs, df_temp], axis=0)
            dfs = dfs.reset_index(drop=True)
        print("number of mol pool:", len(dfs))
        best_model_path = run_step(dfs, epochs, stime)
        MAE, RMSE, R2 , Sp = eval_model(df_test, best_model_path)
        print("MAE:{}, RMSE:{}, R2:{}, Sp:{}, step:{}".format(MAE, RMSE, R2, Sp, stime))

        results["RMSE"].append(RMSE)
        results["R2"].append(R2)
        results["Sp"].append(Sp)

    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(results)

    results_df.to_csv('result/dpp4/ecfp_random_results.csv', index=False)


In [18]:
df = pd.read_csv("preprocess_data/dpp4/dpp4_train.csv", sep="\t")
df_test = pd.read_csv("preprocess_data/dpp4/dpp4_test.csv", sep="\t")
df_test["ecfp"] = df_test["smiles"].map(lambda x: ecfp_from_smiles(x))
df_test = df_test[~df_test["ecfp"].isna()]

In [19]:
run_iterations(df, df_test)

len(df): 24865
number of mol pool: 100
---------------------------------
validation min rmse: 1.9567245969247833
---------------------------------

MAE:1.1815021062021924, RMSE:1.4873911337414645, R2:-0.8382067759741241, Sp:0.467712930999927, step:0
number of mol pool: 200
---------------------------------
validation min rmse: 1.4243525333794398
---------------------------------

MAE:0.9703373654470417, RMSE:1.2315259971414483, R2:-0.26017585466138526, Sp:0.5832321072893527, step:1
number of mol pool: 300
---------------------------------
validation min rmse: 1.5215617756159834
---------------------------------

MAE:1.0282652997512347, RMSE:1.2942528663800565, R2:-0.39181738507102093, Sp:0.5590272741672587, step:2
number of mol pool: 400
---------------------------------
validation min rmse: 1.3563068842702954
---------------------------------

MAE:0.9594000342612311, RMSE:1.2127593493211235, R2:-0.22206202452330026, Sp:0.583040144042161, step:3
number of mol pool: 500
----------------

OSError: ignored